<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_MedmcQADataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Medmcqa.ipynb

In [1]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is in

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [4]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_MedmcQA"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

adapter_config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

The model 'PeftModelForCausalLM' is not supported for document-question-answering. Supported models are ['LayoutLMForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering'].


In [6]:
from datasets import load_dataset
print("Preprocessing dataset Medmcqa")
dataset = load_dataset("openlifescienceai/medmcqa")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Preprocessing dataset Medmcqa


Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/183 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

1910937

In [7]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [9]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['question'][0:nrec]
dataset_final_Answer=dataset['exp'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [10]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [11]:
dataset[10]

{'id': 'b3d1eb6c-3909-4011-ad10-d55538b81456',
 'question': 'Naglers reaction is shown by',
 'opa': 'Clostridium tetani',
 'opb': 'Clostridium botulinum',
 'opc': 'Clostridium perfringens',
 'opd': 'Clostridium septicum',
 'cop': 2,
 'choice_type': 'single',
 'exp': "Nagler's reaction - when Clostridium perfringens is grown in media containing 6 % agar, 5% Fildes peptic digest of sheep blood and 20% human serum, with antitoxin spread on one half of plate, colonies in the other half without antitoxin will be surrounded by a zone of opacity. No opacity around the colonies on the half of plate with antitoxin, this is due to the specific neutralization of the alpha-toxin. This specific lecithinase effect, known as the Nagler's reaction. Reference: Textbook of Microbiology; Anathanarayan and paniker's; 10th edition; Page no: 263",
 'subject_name': 'Microbiology',
 'topic_name': 'Bacteriology'}

In [12]:
datasetF

,id,Question,Answer
0,e9ad821a-c438-4965-9f77-760819dfa155,Chronic urethral obstruction due to benign pri...,Chronic urethral obstruction because of urinar...
1,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,Which vitamin is supplied from only animal sou...,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...
2,5c38bea6-787a-44a9-b2df-88f4218ab914,All of the following are surgical options for ...,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba..."
3,cdeedb04-fbe9-432c-937c-d53ac24475de,Following endaerectomy on the right common car...,The central aery of the retina is a branch of ...
4,dc6794a3-b108-47c5-8b1b-3b4931577249,Growth hormone has its effect on growth through?,"Ans. is 'b' i.e., IGI-1GH has two major functi..."
5,5ab84ea8-12d1-47d4-ab22-668ebf01e64c,Scrub typhus is transmitted by: September 2004,Ans. C i.e. Mite
6,a83de6e4-9427-4480-b404-d96621ebb640,Abnormal vascular patterns seen with colposcop...,"Abnormal vascular pattern include punctation, ..."
7,f3bf8583-231b-4b7a-828c-179b0f9ccdd9,Per rectum examination is not a useful test fo...,PILONIDAL SINUS/DISEASE (Jeep Bottom; Driver's...
8,73515f05-e947-4801-8077-3abdeca95c84,Characteristics of Remifentanyl – a) Metabolis...,Remifentanil is the shortest acting opioid due...
9,53f79833-21b0-4336-8ef4-404c687ec807,Hypomimia is ?,Ans. C. Deficit of expression by gestureHypomi...


In [13]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

In [14]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [15]:
nrec=1
eval_dataset[nrec]

{'id': 'e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3',
 'question': 'Which vitamin is supplied from only animal source:',
 'opa': 'Vitamin C',
 'opb': 'Vitamin B7',
 'opc': 'Vitamin B12',
 'opd': 'Vitamin D',
 'cop': 2,
 'choice_type': 'single',
 'exp': "Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off.",
 'subject_name': 'Biochemistry',
 'topic_name': 'Vitamins and Minerals'}

In [16]:
eval_dataset[nrec]['question']

'Which vitamin is supplied from only animal source:'

In [17]:
eval_dataset[nrec]['exp']

"Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off."

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [97]:
from random import randint
rand_idx = randint(0, len(eval_dataset))

rand_idx = 2
#,3600, 5570,10077 ## good answer

In [98]:
rand_idx

2

In [48]:
#rand_idx = randint(0, len(eval_dataset))
#modified_conversation = [
#    {"role": "user", "content": eval_dataset[rand_idx]['Question']},
#    {"role": "assistant", "content": eval_dataset[rand_idx]['Answer']}
#]

In [99]:
# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])

#Use the chat template to format the input for the language model
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = eval_dataset[rand_idx]['question']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [76]:
outputs

[{'generated_text': 'Scrub typhus is transmitted by: September 2004 [/INST] Ans. D: Mite Sarcoptes scabiei var hominis Mite S. scabiei var hominis is a common human parasite and the etiologic agent of scabies, a pruritic cutaneous disease Scabies is a skin disease caused by a mite, Sarcoptes scabiei var hominis. The mite burrows into the stratum corneum of the epidermis. The disease is transmitted by close contact with infected persons or by contact with contaminated objects or bedding. Scabies is'}]

In [100]:
oanswer=eval_dataset[rand_idx]['exp']
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
#rand_idx = randint(0, len(eval_dataset))

print(f"Question:\n{eval_dataset[rand_idx]['question']}")
print()
print(f"Original Answer:\n{eval_dataset[rand_idx]['exp']}")
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+9:].strip()}")

from difflib import SequenceMatcher

  # Utility function to compute similarity
def similar(str1, str2):
  return SequenceMatcher(None, str1, str2).ratio()

# using SequenceMatcher.ratio()

# similarity between strings
similarity = similar(oanswer, ganswer)

# printing the result
print()
print ("The similarity between 2 strings is : " + str(similarity))



Question:
All of the following are surgical options for morbid obesity except -

Original Answer:
Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastroplasty & Adjustable gastric bandingo Malabsorbtive procedures include Biliopancreatic diversion, and Duodenal switcho Roux-en Y gastric bypass has features of both restriction and malabsorptionBariatric Operations: Mech

In [52]:
prompt='capital of russia?'

outputs = generation_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7,
                              top_k=50, top_p=0.9, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [53]:
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

Generated Answer:
Moscow is the capital and largest city of Russia.It is on the Moskva River in the northwestern part of the country, approximately 300 kilometres (190 mi) east of the Baltic Sea and 300 kilometres (190 mi) northwest of the Black Sea. Moscow is the political, economic, cultural, scientific and religious capital of Russia. Moscow has a population of over 12 million.


In [54]:
q=eval_dataset[1]['question']

In [55]:
number_of_eval_samples = 10
eval_dataset.shuffle().select(range(number_of_eval_samples))

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 10
})

In [56]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [ ]:
from tqdm import tqdm
import re

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")

def evaluate(sample):

    prompt = sample['question']
    answer=sample['exp']

    if answer == None:
       answer_original = 'Not possible to get or use'

    answer_original = re.sub("\n", "", str(answer))


    if len(answer_original)==0:
        answer_original='Not possible to get or use'

    str_question= sample['question']

    prompt = re.sub("\n", "", str_question)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+9:].strip()
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

    oanswer=sample['exp']

    if oanswer == None:
       oanswer = 'Not possible to get or use'

    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
    #print()
    #print(f"Question:\n{sample['question']}")
    #print(f"Original Answer:\n{sample['exp']}")
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+9:].strip()}")
    #print()

    # Python3 code to demonstrate
    # similarity between strings
    # using SequenceMatcher.ratio()
    from difflib import SequenceMatcher

    # Utility function to compute similarity
    def similar(str1, str2):
      return SequenceMatcher(None, str1, str2).ratio()

    # using SequenceMatcher.ratio()
    # similarity between strings
    res = similar(oanswer, ganswer)

    # printing the result
    #print ("The similarity between 2 strings is : " + str(res))



    if  res>=0.15:
          #print(f"Question:\n{sample['Question']}")
          #print(f"Original Answer:\n{answer_original}")
          #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")
          return 1
    else:
          #print(f"Original Answer:\n{answer_original}")
          return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict

for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
#for n in range(number_of_eval_samples):
#    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

 97%|█████████▋| 971/1000 [1:21:17<01:49,  3.79s/it]